In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from datetime import datetime

In [ ]:
def reverse_date(btc, remove_date="Yes"):
    '''Reverses the dataset so it is in chronological order. Optional to remove date column and set as index.'''
    final_data = btc.reindex(index=btc.index[::-1])
    if remove_date == "Yes":
        # Need to reverse order of dataframe.
        final_data = final_data.set_index(final_data['Date'])
        del final_data['Date']
    return final_data

In [ ]:
btc = pd.read_csv('https://raw.githubusercontent.com/chrishyland/Crypto-position/master/bitcoin_final.csv')
crix = pd.read_csv('https://raw.githubusercontent.com/chrishyland/Crypto-position/master/crix.csv')

df = reverse_date(btc)
df.head()

In [ ]:
def ichimoku_plot(df):
    '''Computes the Ichimoku Kinkō Hyō trend identification system.'''
    # This plot has 5 components to it. 
    high_prices = df['High']
    close_prices = df['Close']
    low_prices = df['Low']
    dates = df.index
    
    # Ichimoku (Conversion Line): (9-period high + 9-period low)/2
    nine_period_high = df['High'].rolling(window=9, center=False).max() # Usually window is 9 days.
    nine_period_low = df['Low'].rolling(window=9, center=False).max() 
    ichimoku = (nine_period_high + nine_period_low) /2
    df['tenkan_sen'] = ichimoku
    
    # Kijun-Sen (Base line): (26-period high + 26-period low)/2)
    period26_high = high_prices.rolling(window=26, center=False).max() # Window normally 26 days.
    period26_low = low_prices.rolling(window=26, center=False).min()
    df['kijun_sen'] = (period26_high + period26_low) / 2
    
    # Senkou Span A (Leading span A): (Base line + Conversion line) / 2
    df['senkou_span_a'] = ((df['tenkan_sen'] + df['kijun_sen']) / 2).shift(26) 

    # Senkou Span B (Leading span B): (52-period high + 52-period low)/2
    period52_high = high_prices.rolling(window=52, center=False).max()
    period52_low = low_prices.rolling(window=52, center=False).min()
    df['senkou_span_b'] = ((period52_high + period52_low) / 2).shift(26)
    
    # Chikou Span (Lagging span): Closing price of last 22 periods
    df['chikou_span'] = close_prices.shift(-22)
    
    return df[df.columns[6:]]

In [ ]:
def sma_plot(df, window):
    '''Computes simple moving average.'''
    rolling = df['Close'].rolling(window=window) # Window tells us how many days average to take.
    return rolling.mean()

In [ ]:
def bollinger_plot(df, window, num_sd):
    '''Computes Bollinger bands depending on number of standard deviation and window.''' 
    rolling_mean = df['Close'].rolling(window).mean() # Window should be same as SMA.
    rolling_std = df['Close'].rolling(window).std()
    
    bollinger = pd.DataFrame(data=None)
    bollinger['Rolling Mean'] = rolling_mean
    bollinger['Bollinger High'] = rolling_mean + (rolling_std * num_sd)
    bollinger['Bollinger Low'] = rolling_mean - (rolling_std * num_sd)

    return bollinger

In [ ]:
# TODO: Configure Bokeh to allow for toggle configuration for sliding bar. Allows us to choose the date.
# Example: https://finance.yahoo.com/chart/AAPL#eyJpbnRlcnZhbCI6ImRheSIsInBlcmlvZGljaXR5IjoxLCJ0aW1lVW5pdCI6bnVsbCwiY2FuZGxlV2lkdGgiOjgsInZvbHVtZVVuZGVybGF5Ijp0cnVlLCJhZGoiOnRydWUsImNyb3NzaGFpciI6dHJ1ZSwiY2hhcnRUeXBlIjoibGluZSIsImV4dGVuZGVkIjpmYWxzZSwibWFya2V0U2Vzc2lvbnMiOnt9LCJhZ2dyZWdhdGlvblR5cGUiOiJvaGxjIiwiY2hhcnRTY2FsZSI6ImxpbmVhciIsInBhbmVscyI6eyJjaGFydCI6eyJwZXJjZW50IjoxLCJkaXNwbGF5IjoiQUFQTCIsImNoYXJ0TmFtZSI6ImNoYXJ0IiwidG9wIjowfX0sInNldFNwYW4iOnt9LCJsaW5lV2lkdGgiOjIsInN0cmlwZWRCYWNrZ3JvdWQiOnRydWUsImV2ZW50cyI6dHJ1ZSwiY29sb3IiOiIjMDA4MWYyIiwic3ltYm9scyI6W3sic3ltYm9sIjoiQUFQTCIsInN5bWJvbE9iamVjdCI6eyJzeW1ib2wiOiJBQVBMIn0sInBlcmlvZGljaXR5IjoxLCJpbnRlcnZhbCI6ImRheSIsInRpbWVVbml0IjpudWxsLCJzZXRTcGFuIjp7fX1dLCJzdHVkaWVzIjp7InZvbCB1bmRyIjp7InR5cGUiOiJ2b2wgdW5kciIsImlucHV0cyI6eyJpZCI6InZvbCB1bmRyIiwiZGlzcGxheSI6InZvbCB1bmRyIn0sIm91dHB1dHMiOnsiVXAgVm9sdW1lIjoiIzAwYjA2MSIsIkRvd24gVm9sdW1lIjoiI0ZGMzMzQSJ9LCJwYW5lbCI6ImNoYXJ0IiwicGFyYW1ldGVycyI6eyJoZWlnaHRQZXJjZW50YWdlIjowLjI1LCJ3aWR0aEZhY3RvciI6MC40NSwiY2hhcnROYW1lIjoiY2hhcnQifX0sIuKAjEJvbGxpbmdlciBCYW5kc%2BKAjCAoQywxMCwyLG1hLHkpIjp7InR5cGUiOiJCb2xsaW5nZXIgQmFuZHMiLCJpbnB1dHMiOnsiRmllbGQiOiJDbG9zZSIsIlBlcmlvZCI6IjEwIiwiU3RhbmRhcmQgRGV2aWF0aW9ucyI6MiwiTW92aW5nIEF2ZXJhZ2UgVHlwZSI6InNpbXBsZSIsIkNoYW5uZWwgRmlsbCI6dHJ1ZSwiaWQiOiLigIxCb2xsaW5nZXIgQmFuZHPigIwgKEMsMTAsMixtYSx5KSIsImRpc3BsYXkiOiLigIxCb2xsaW5nZXIgQmFuZHPigIwgKEMsMTAsMixtYSx5KSJ9LCJvdXRwdXRzIjp7IkJvbGxpbmdlciBCYW5kcyBUb3AiOiIjZmZkYjQ4IiwiQm9sbGluZ2VyIEJhbmRzIE1lZGlhbiI6IiNmZmEzM2YiLCJCb2xsaW5nZXIgQmFuZHMgQm90dG9tIjoiI2ZmYmQ3NCJ9LCJwYW5lbCI6ImNoYXJ0IiwicGFyYW1ldGVycyI6eyJjaGFydE5hbWUiOiJjaGFydCJ9fX19

In [ ]:
from bokeh.plotting import figure, output_file, show


In [ ]:
# output to static HTML file
output_file("lines.html")

# create a new plot with a title and axis labels
p = figure(title="SMA of bitcoin", x_axis_label='Date', y_axis_label='Price')

# add a line renderer with legend and line thickness
p.line(df.index, df.High, legend="Temp.", line_width=2)

# show the results
show(p)

In [ ]:
import plotly
plotly.tools.set_credentials_file(username='chrishigh95', api_key='Mg0N9OIjeBB64gnsBP1Y')
import plotly.plotly as py
from plotly.graph_objs import *

In [ ]:
data = [Bar(x=df.index,
            y=sma_plot(df, 9))]

py.iplot(data)

In [ ]:
import numpy as np

import plotly.plotly as py

# Create a trace
trace = go.Ohlc(
    open = dt['Open'],
    high = dt['High'],
    low = dt['Low'],
    close = dt['Close']
)
data = [trace]

layout = dict(
    title = "BTC USD Cross"
)
fig = dict(data=data, layout=layout)

py.iplot(fig, filename='basic-line')

In [ ]:
boll_low = go.Scatter(
    x=dt.index,
    y=dt['Bollinger High'],
    name = "Bollinger High",
    line = dict(color = '#66c2ff'),
    opacity = 0.8)

boll_high = go.Scatter(
    x=dt.index,
    y=dt['Bollinger Low'],
    name = "Bollinger Low",
    line = dict(color = '#99d6ff'),
    opacity = 0.8)
close = go.Scatter(
    x=dt.index,
    y=dt['Close'],
    name = "Close",
    line = dict(color = '#006bb3')

)

data1 = [boll_low, boll_high, close]
layout1 = dict(
    title='Bollinger bands',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(range="28/4/2013 - 22/11/2017"),
        type='date'
    )
)

fig = dict(data=data1, layout=layout1)

py.iplot(fig, filename='basic-line2')